In [14]:
import cv2
import os
import numpy as np

def detect_aruco_markers_and_save(image_path, save_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(image, (5,5), 0)
    
    aruco_dict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_ARUCO_ORIGINAL)
    parameters = cv2.aruco.DetectorParameters_create()
    corners, ids, rejected_img_points = cv2.aruco.detectMarkers(gray, aruco_dict, parameters=parameters)
    image_with_markers = cv2.aruco.drawDetectedMarkers(image.copy(), corners, ids, (0, 255, 0))
    height, width, _ = image.shape
    save_filename = os.path.join(save_path, 'detected_markers_image.jpg')
    cv2.imwrite(save_filename, image_with_markers)
    print(f"Image with detected markers saved at: {save_filename}")

    
    center_points = []
    for (marker, id) in zip(corners, ids):
        corner = marker.reshape((4, 2))
        (tl, tr, br, bl) = corner
        cx = int((tl[0] + br[0]) / 2)
        cy = int((tl[1] + br[1]) / 2)
        center_points.append((cx, cy))
        cv2.circle(image, (cx, cy), 4, (255, 1, 1), 2)  
        label_text = f'ID:{id[0]} ({cx},{cy})'
        cv2.putText(image, label_text, (cx + 2, cy - 1), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (1, 1, 1), 1, cv2.LINE_AA)

    center_points = np.array(center_points)
    hull = cv2.convexHull(center_points, clockwise=True, returnPoints=False)
    rect = cv2.minAreaRect(center_points)
    box = cv2.boxPoints(rect)
    box = np.int0(box)
    area = cv2.contourArea(box)
    

    (center, size, angle) = rect
    (width, height) = size
    print(center)
    
    print(f"Length of the rectangle: {width} pixels")
    print(f"Breadth of the rectangle: {height} pixels")

    mask = np.zeros_like(image)
    cv2.drawContours(mask, [box], 0, (255, 255, 255), 1)
    result_cropped = cv2.bitwise_and(image, mask)

    center = (int(rect[0][0]), int(rect[0][1]))
    radius = 50
    cv2.circle(image, center, radius, (123, 222, 1), 5)
    print(area)
    
    box_image = np.zeros_like(image)
    cv2.drawContours(box_image, [box], 0, (122,250, 243), -1)

    opacity = 0.10
    cv2.addWeighted(box_image, opacity, image, 1 - opacity, 0, image)
    
    edges = cv2.Canny(gray, 10, 150)

    # Find contours in the edged image
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Iterate through the contours and find the center of each block
    for contour in contours:
        # Calculate the center of the contour
        M = cv2.moments(contour)
        if M["m00"] != 0:
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
    
            # Draw a circle at the center of the block
            cv2.circle(image, (cX, cY), 5, (0, 255, 0), -1)
    
            # Annotate the center coordinates
            text = f"({cX}, {cY})"
            cv2.putText(image, text, (cX - 50, cY - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)



    # Save the cropped image
    save_filename_cropped = os.path.join(save_path, 'cropped_image.jpg')
    cv2.imwrite(save_filename_cropped, result_cropped)
    print(f"Cropped image inside the resultant box saved at: {save_filename_cropped}")
    save_filename_with_circles_and_box = os.path.join(save_path, 'image_with_circles_box.jpg')
    cv2.imwrite(save_filename_with_circles_and_box, image)
    print(f"Image with circles, blue box, and lines saved at: {save_filename_with_circles_and_box}")

image_path = '/home/qt/visionSort/visionSort_cv/imgs/test.jpg'
save_path = '/home/qt/visionSort/visionSort_cv/imgs'
detect_aruco_markers_and_save(image_path, save_path)

Image with detected markers saved at: /home/qt/visionSort/visionSort_cv/imgs/detected_markers_image.jpg
(1901.69189453125, 1017.8702392578125)
Length of the rectangle: 1156.2728271484375 pixels
Breadth of the rectangle: 1647.0194091796875 pixels
1903980.0
Cropped image inside the resultant box saved at: /home/qt/visionSort/visionSort_cv/imgs/cropped_image.jpg
Image with circles, blue box, and lines saved at: /home/qt/visionSort/visionSort_cv/imgs/image_with_circles_box.jpg
